# How we made the model
**Step 1**: Find and scrape data

First we found two websites that we were going to scrape to try and get data.
- https://capitol.texas.gov
- https://lrl.texas.gov

There were two main steps to scrape all the necessary data. We needed to get data about individual bills, and we needed to get data about legislative sessions. 

## Scraping Bill data
All bill data was scraped in 'bill_data_scraper.py'. This script is ran through the terminal with 'python bill_data_scraper.py'. You then give it which bill, session, and chamber of congress you want it to start scraping from. This allows you to have multiple instances of the script running, allowing you to scrape the data much faster than just one script.

### How The Script Works
The script starts by looking up each bill at 'https://capitol.texas.gov/BillLookup/History.aspx?LegSess={}&Bill={}{}' The script goes through all bills in that section, and when it fails to find a bill 10 times, then it moves on to the next chamber or legSess. 

In [1]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
if soup.text.find('The bill number does not exist for the selected legislative session.') != -1:
        if times_in_row_failed_to_find_bill < 10:
            times_in_row_failed_to_find_bill+=1
            bill_number+=1
            continue
        
        times_in_row_failed_to_find_bill = 0

        #save to csv
        if df is not None:
            end = time.time()
            print('session {}, {}, complete through bill {} --  time: {}'.format(leg_sessions[leg_session_index], bill_name, bill_number - 11, end - start))
            if not path.isfile('bill_data.csv'):
                df.to_csv('bill_data.csv', mode='a',index=False)
            else:
                df.to_csv('bill_data.csv', mode='a',index=False, header=False)
            df = None

            start = time.time()

        if bill_name == 'HB':
            bill_name = 'SB'
            bill_number = 1

        else:
            leg_session_index+=1
            bill_number = 1
            bill_name = 'HB'

        continue

SyntaxError: 'continue' not properly in loop (1122528538.py, line 6)

Assuming we could find the bill data, we use requests and beautiful soup to extract the data from each bill. Requests allows us to get the HTML from the page, and beautiful makes it easy to parse through all the data and extract what we need

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
import requests
from bs4 import BeautifulSoup
URL = 'https://capitol.texas.gov/BillLookup/History.aspx?LegSess={}&Bill={}{}'.format(leg_sessions[leg_session_index],bill_name,bill_number)
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

We ended up scraping data from :
- https://capitol.texas.gov/BillLookup/History.aspx?LegSess={}&Bill={}{}
- https://capitol.texas.gov/BillLookup/Authors.aspx?LegSess={}&Bill={}{}
- https://capitol.texas.gov/BillLookup/Actions.aspx?LegSess={}&Bill={}{}


All bill data was stored in a dictionary named 'bill'. The code below is an example on how we scraped data from the history page, this process is repeated from all the links above, which in total gives us all the data we scraped from https://capitol.texas.gov.

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
#get bill name details
billName = soup.find(id="usrBillInfoTabs_lblBill").text
bill["house_bill"] = billName.find("HB") != -1
bill["senate_bill"] = billName.find("SB") != -1
bill["bill_number"] = int(billName[3:])

#get bill legislative session
bill["legislative_session"] = leg_sessions[leg_session_index]

#get house committee
house_committee = soup.find(id="cellComm1Committee")
if house_committee:
    bill["house_committee"] = house_committee.text

#get senate committee
senate_committee = soup.find(id="cellComm2Committee")
if senate_committee:
    bill["senate_committee"] = senate_committee.text

#get subjects
subjects = soup.find(id="cellSubjects")
if subjects:
    bill["subjects"] = subjects.contents[::2]

#get sponsor
sponsor = soup.find(id="cellSponsors")
if sponsor:
    bill["sponsor"] = sponsor.text

### How we saved the data
All the bill data was saved into 'bill_data.csv'. This csv is just a huge excel sheet of all the data about each bill. Before the data is saved in the spreadsheet some data is concatenated from 'leg_sess_data.csv', but I'll go how we got that data next. Once the data about the legSess is added the following shows how each bill is added as a row in a dataframe, and then how that dataframe is added to the end of the csv. Data is added to the end of the csv after 100 bills, this way the program isn't slowed down by writing to the csv after every bill, but still saves incrementaly incase there was a problem

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON

#add to df
if df is None:
    df = pd.DataFrame(np.array([list(bill_data_for_csv.values())]), columns=list(bill_data_for_csv.keys()))
else:
    df2 = pd.DataFrame(np.array([list(bill_data_for_csv.values())]), columns=list(bill_data_for_csv.keys()))
    df = pd.concat([df, df2])



#save to csv
if df is not None:
    if bill_number % 100 == 0:
        end = time.time()
        print('session {}, {}, {} - {} complete  --  time: {}'.format(leg_sessions[leg_session_index], bill_name, bill_number - 100, bill_number, end - start))
        if not path.isfile('bill_data.csv'):
            df.to_csv('bill_data.csv', mode='a',index=False)
        else:
            df.to_csv('bill_data.csv', mode='a',index=False, header=False)
        df = None
        start = time.time()

bill_number+=1

## Scraping Legislative Session Data

All legislative session data was scraped from 'https://lrl.texas.gov' and saved into 'leg_sess_data.csv'. 'bill_data_scraper.py' used 'leg_sess_data.csv' to complete its information about each bill.

Every bill introduced into the texas legislature came from a legislative session. These legislative sessions have a certain number of democrats, republicans, young congressmen/women, and old congressmen/women. 'legislative_session_data_scraper' is the script that scraped the data from 'https://lrl.texas.gov' and saved into 'leg_sess_data.csv'.


### How the Script Works
Once again we use requests and beautiful soup to parse html pages and gain information about the legislative session.
The exact pages used to extract data are:
- 'https://lrl.texas.gov/sessions/memberStatistics.cfm'
- 'https://lrl.texas.gov/committees/cmtes.cfm?from=session&session={}'
- 'https://lrl.texas.gov/'
- 'https://lrl.texas.gov/committees/{}'

The script reads from general summary pages and also reads through committe pages, then looks at each committee members politcal party and aggregates all that data into 'leg_sess_data.csv'

## Combinging Legislative Session Data into 'bill_data_scraper.py'

The legSess data from 'leg_sess_data.csv' is combined will individual bill information in 'bill_data_scraper.py' through the following code

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
#get session information
leg_sess_df = pd.read_csv('leg_sess_data.csv')
session_data = leg_sess_df.loc[leg_sess_df['leg_sess'] == int(leg_sessions[leg_session_index][:2])]

bill["male_house_members"] = int(session_data.iloc[0]['male_house_members'])
bill["male_senate_members"] = int(session_data.iloc[0]['male_senate_members'])
bill["female_house_members"] = int(session_data.iloc[0]['female_house_members'])
bill["female_senate_members"] = int(session_data.iloc[0]['female_senate_members'])
bill["democrat_house_members"] = int(session_data.iloc[0]['democrat_house_members'])
bill["democrat_senate_members"] = int(session_data.iloc[0]['democrat_senate_members'])
bill["republican_house_members"] = int(session_data.iloc[0]['republican_house_members'])
bill["republican_senate_members"] = int(session_data.iloc[0]['republican_senate_members'])
bill["house_incumbents"] = int(session_data.iloc[0]['house_incumbents'])
bill["senate_incumbents"] = int(session_data.iloc[0]['senate_incumbents'])
bill["house_freshman"] = int(session_data.iloc[0]['house_freshman'])
bill["senate_freshman"] = int(session_data.iloc[0]['senate_freshman'])
bill["house_members_age_under_30"] = int(session_data.iloc[0]['house_members_age_under_30'])
bill["senate_members_age_under_30"] = int(session_data.iloc[0]['senate_members_age_under_30'])
bill["house_members_age_30_to_39"] = int(session_data.iloc[0]['house_members_age_30_to_39'])
bill["senate_members_age_30_to_39"] = int(session_data.iloc[0]['senate_members_age_30_to_39'])
bill["house_members_age_40_to_49"] = int(session_data.iloc[0]['house_members_age_40_to_49'])
bill["senate_members_age_40_to_49"] = int(session_data.iloc[0]['senate_members_age_40_to_49'])
bill["house_members_age_50_to_59"] = int(session_data.iloc[0]['house_members_age_50_to_59'])
bill["senate_members_age_50_to_59"] = int(session_data.iloc[0]['senate_members_age_50_to_59'])
bill["house_members_age_60_to_69"] = int(session_data.iloc[0]['senate_members_age_60_to_69'])
bill["house_members_age_over_70"] = int(session_data.iloc[0]['house_members_age_over_70'])
bill["senate_members_age_over_70"] = int(session_data.iloc[0]['senate_members_age_over_70'])

bill["house_committee_democrats"] = int(session_data.iloc[0]['committee_{}_democrats'.format(committee_to_number.get(bill["house_committee"],0))])
bill["house_committee_republicans"] = int(session_data.iloc[0]['committee_{}_republicans'.format(committee_to_number.get(bill["house_committee"],0))])
bill["senate_committee_democrats"] = int(session_data.iloc[0]['committee_{}_democrats'.format(committee_to_number.get(bill["senate_committee"],0))])
bill["senate_committee_republicans"] = int(session_data.iloc[0]['committee_{}_republicans'.format(committee_to_number.get(bill["senate_committee"],0))])


#translate data to what were storing in the csv
bill_data_for_csv = {
    "house_bill": bill["house_bill"],
    "senate_bill": bill["senate_bill"],
    "bill_number": bill["bill_number"],
    "legislative_session": bill["legislative_session"],
    #"party_of_primary_author": None,
    "num_of_joint_authors": len(bill['joint_authors']),
    "num_of_co_authors": len(bill["co_authors"]),
    "num_of_subjects": len(bill["subjects"]),
    #"party_of_sponsor": None,
    "passed": bill["passed"],
    "male_house_members": bill["male_house_members"],
    "male_senate_members": bill["male_senate_members"],
    "female_house_members": bill["female_house_members"],
    "female_senate_members": bill["female_senate_members"],
    "democrat_house_members": bill["democrat_house_members"],
    "democrat_senate_members": bill["democrat_senate_members"],
    "republican_house_members": bill["republican_house_members"],
    "republican_senate_members": bill["republican_senate_members"],
    "house_incumbents": bill["house_incumbents"],
    "senate_incumbents": bill["senate_incumbents"],
    "house_freshman": bill["house_freshman"],
    "senate_freshman": bill["senate_freshman"],
    "house_members_age_under_30": bill["house_members_age_under_30"],
    "senate_members_age_under_30": bill["senate_members_age_under_30"],
    "house_members_age_30_to_39": bill["house_members_age_30_to_39"],
    "senate_members_age_30_to_39": bill["senate_members_age_30_to_39"],
    "house_members_age_40_to_49": bill["house_members_age_40_to_49"],
    "senate_members_age_40_to_49": bill["senate_members_age_40_to_49"],
    "house_members_age_50_to_59": bill["house_members_age_50_to_59"],
    "senate_members_age_50_to_59": bill["senate_members_age_50_to_59"],
    "house_members_age_60_to_69": bill["house_members_age_60_to_69"],
    "house_members_age_over_70": bill["house_members_age_over_70"],
    "senate_members_age_over_70": bill["senate_members_age_over_70"],
    "house_committee_democrats": bill["house_committee_democrats"],
    "house_committee_republicans": bill["house_committee_republicans"],
    "senate_committee_democrats": bill["senate_committee_democrats"],
    "senate_committee_republicans": bill["senate_committee_republicans"]
}

## Committee Names Debacle

If you read through the code you might be wondering about 'committee_dict.py', 'committee_mapping.py', and 'committee_names.txt'. These exist because in https://capitol.texas.gov and https://lrl.texas.gov the committee names are slightly different. For example the same committee might be called 'Appropriations - S/C on Budget Transparency and Reform' on one website, and 'Appropriations Subcommittee on Budget Transparency and Reform' on the other. My solution to this was to assign each committe a number, and then assign each committee name to the respective number. So 'Appropriations - S/C on Budget Transparency and Reform' and 'Appropriations Subcommittee on Budget Transparency and Reform' both represent committee 410.

To make this mapping from the different websites I used the scripts and files 'committee_mapping.py', 'committee_names.txt', and 'commitee_dict.py'. You can read through those scripts and hopefully everything makes since.

## The Overall Data

Overall the data for each bill that was scraped using the above method are:
- house_bill
- senate_bill
- bill_number
- legislative_session
- num_of_joint_authors
- num_of_co_authors
- num_of_subjects
- passed
- male_house_members
- male_senate_members
- female_house_members
- female_senate_members
- democrat_house_members
- democrat_senate_members
- republican_house_members
- republican_senate_members
- house_incumbents
- senate_incumbents
- house_freshman
- senate_freshman
- house_members_age_under_30
- senate_members_age_under_30
- house_members_age_30_to_39
- senate_members_age_30_to_39
- house_members_age_40_to_49
- senate_members_age_40_to_49
- house_members_age_50_to_59
- senate_members_age_50_to_59
- house_members_age_60_to_69
- house_members_age_over_70
- senate_members_age_over_70
- house_committee_democrats
- house_committee_republicans
- senate_committee_democrats
- senate_committee_republicans

### Cleaning up the data and csv

Because 'bill_data_scraper.py' can have multiple instances running at a time, the 'bill_data.csv' isn't organized in order. In order to sort this 'bill_data_cleanup.ipynb' was created. By running this script all that happens is it sorts the csv by name so it was easier to tell what data needed to be mined next

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
import pandas as pd
import numpy as np

df = pd.read_csv('bill_data.csv')

df = df.sort_values(by=['legislative_session', 'senate_bill', 'bill_number'])
df.to_csv('bill_data.csv', mode='w',index=False)

One more script was used to clean up the data, 'removing_errors_from_data.ipynb'. There was an issue with house bills have senate committee data, and senate bills having house committee data. This data exists, but we don't want to use it because a bill with senate and house committee data must have passed at least one chamber of congress, meaning it is extremely likely to pass. The first model was way to accurate because of this, so 'removing_errors_from_data.ipynb' fixes this problem for us

## Building the Model

Now that we have all the data we just need to build the model. The model was built using the data in bill_data.csv. The model is built in 'building_the_fucking_model.ipynb'
Some features need to be removed from the csv before we feed it into the model. Specifically
- bill_number
- legislative_session
- house_bill
- senate_bill
- passed

After we remove these columns we then split the data into training and testing data

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
import pandas as pd

df = pd.read_csv('bill_data.csv')

df.head()

X = df.copy()
X.pop('bill_number')
X.pop('legislative_session')
X["house_bill"] = X["house_bill"].astype(int)
X["senate_bill"] = X["senate_bill"].astype(int)
X["passed"] = X["passed"].astype(int)
y = X.pop('passed')

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.75)
input_shape = [X_train.shape[1]]

Then we make a simple deep learning model using tensorflow and keras. We use some batch normilization and dropout with early stopping to prevent overfitting

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.3),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.3),
    
    layers.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping]
)
model.save('./the_model')

## Evaluating the model
Then after the model is made there are two more code blocks in 'building_the_fucking_model.ipynb' used to evaluate the model.
The code below is used to evaluate any specific bill in the csv

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
import pandas as pd
testDF = pd.read_csv('bill_data.csv')
T = testDF.copy()
T["house_bill"] = T["house_bill"].astype(int)
T["senate_bill"] = T["senate_bill"].astype(int)
T["passed"] = T["passed"].astype(int)

#legislative_session = "82R" #house_bill 472
T = T.loc[(T['legislative_session'] == "82R") & (T['house_bill'] == 1) & (T['bill_number'] == 472)]
T.pop('bill_number')
T.pop('legislative_session')
T.pop('passed')

yhat = model.predict(T)
print(yhat)


Then to evaluate to model overall the following code makes a confusion matrix

In [ ]:
#THIS CODE IS NOT MEANT TO BE RUN, IT IS JUST HERE FOR YOU TO READ AND UNDERSTAND WHAT IS GOING ON
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

y_pred = model.predict(X_valid)
y_pred = [1 if i > .5 else 0 for i in y_pred]
cm = confusion_matrix(y_valid, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()


## Using the web app
The web app uses a react front end and a python flask back end. To run the back end server you simply need to type 'python server.py' front the 'back_end' directory and it starts the server and reads the model from 'the_model' directory.
In order to run the react front end you need to be in the 'front_end' directory and type 'npm start' in the terminal. The back_end wont work if you dont install the files in requirements.txt and the front end wont run if you dont have Node.js or the Node dependicies installed.